# Imports

In [1]:
import re
import numpy as np
from collections import Counter
import pandas as pd
import itertools
import time

# Code to read data

In [2]:
def read_file(filename):
    with open(filename, 'r') as file:
        content = file.read()  # Open the file in read mode

    data = {}  # Initialize data as an empty dictionary

    # Each file has number of inputs under which data is stored as a matrix
    numerical_data = re.findall(r'\[([\d\s.]+)\]', content)  # Extract values following each header as numerical values and remove '\t's and '\n's

    sections = ['Orders', 'Allocations', 'DistanceShelfShelf', 'DistancePackagingShelf', 'FullDistanceMatrixMetres']

    for i, section in enumerate(sections):
        data[section] = []
        lines = numerical_data[i].strip().split('\n')
        for line in lines:
            values = line.strip().split()  # Split using spaces
            data[section].append([int(val) for val in values])  # Assuming all values are integers

    return data

In [3]:
filename = r"Xpress_Data_Files/Data_Xpress_FullDist_Metres.txt"
data = read_file(filename)

# Extracting data into arrays
Orders = data.get('Orders')
Allocations = np.asarray(data.get('Allocations')[0])
DistanceShelfShelf = np.asarray(data.get('DistanceShelfShelf'))
FullDistanceMatrix = np.asarray(data.get('FullDistanceMatrixMetres'))
FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 1)
FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 0)

# Initialise number of shelves and shelf numbers
NbShelves = 96
Shelves = range(1, NbShelves + 1)


# Functions

In [4]:
def find_closest_product(current_shelf, order, distances):
    '''
    Find the closest shelf which contains a product in the order from the current shelf
    '''
    closest_shelf = None
    min_distance = float(1e6)
    for shelf in order:
        if distances[current_shelf][shelf] <= min_distance:
            closest_shelf = shelf
            min_distance = distances[current_shelf][shelf]
    return closest_shelf

def generate_order_lists(order):
    ''' 
    Takes a list with tuple elements and returns a list of lists 
    with all possible combinations of individual elements individual elements.
    '''
    order_lists = []
    tuple_indices = [i for i, item in enumerate(order) if isinstance(item, tuple)]
    for combination in itertools.product(*[order[i] for i in tuple_indices]):
        new_order = order.copy()
        for i, index in enumerate(tuple_indices):
            new_order[index] = combination[i]
        order_lists.append(new_order)
    return order_lists

def greedy_order_route(order, distances):
    ''' 
    Uses a greedy method of calculating the minimum distance. 

    Function has been split into two if statements to consider
    cases of orders where products are contained on more than one shelf.

    If products are contained on more than one shelf, the function constructs 
    a route with all possible shelf combinations and chooses the one with 
    the shortest distance.

    Returns a list containing the route and the total distance for the order.
    '''

    # If all products in the order are contained on one shelf only
    if not any(isinstance(product, tuple) for product in order):
        visited = [0]
        current_position = 0  
        for k in range(len(order)):
                closest_product = find_closest_product(current_position, order, distances)
                visited.append(closest_product)
                order.remove(closest_product)
                current_position = closest_product
        visited.append(0)
        OrderDistance = 0
        for i in range(len(visited) - 1):
            OrderDistance += distances[visited[i]][visited[i+1]]
        order_distance_final = OrderDistance
        visited_final = visited

    # If one or more products in the order are contained on more than one shelf
    elif any(isinstance(product, tuple) for product in order):
        order_combinations = generate_order_lists(order) # create new orders with all possible combinations from tuples
        order_routes = []   # initialise a list of routes for all combinations                                 
        order_distances = [] # initialise a list of distances for all combinations
        
        # loop over all combinations
        for order in order_combinations:
            visited = [0]
            current_position = 0  
            for k in range(len(order)):
                closest_product = find_closest_product(current_position, order, distances)
                visited.append(closest_product)
                order.remove(closest_product)
                current_position = closest_product
            visited.append(0)
            order_routes.append(visited) # add the route for the combination to the list of routes
            OrderDistance = 0
            for i in range(len(visited) - 1):
                OrderDistance += distances[visited[i]][visited[i+1]]
            order_distances.append(OrderDistance) # add the distance for the combination to the list of distances
        
        # select the order with the shortest distance among the combinations
        min_idx = order_distances.index(min(order_distances))
        visited_final = order_routes[min_idx] 
        order_distance_final = order_distances[min_idx]
            
    return visited_final, order_distance_final # return order route and distance

def convert_orders_to_shelf_indices(allocations):
    ''' 
    This function takes the allocation vector and returns an 
    order matrix with shelf indices instead of product indices.
    '''
    product_to_shelf = {}
    for shelf_index, product in enumerate(allocations):
        if product != 0:  # Check if the element is not zero
            if product not in product_to_shelf:
                product_to_shelf[product] = [shelf_index + 1]  # Initialize with a list containing the current shelf index
            else:
                # If the product already exists in the dictionary, append the new shelf index to the list
                product_to_shelf[product].append(shelf_index + 1)

    # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
    product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

    OrdersByShelf = []
    for order in Orders:
        order_shelf_indices = []
        for product_index in order:
            if product_index in product_to_shelf_tuples:
                shelf_indices = product_to_shelf_tuples[product_index]
                order_shelf_indices.append(shelf_indices)
            else:
                order_shelf_indices.append(0)  # Product not found in allocation matrix
        OrdersByShelf.append(order_shelf_indices)

    return OrdersByShelf

In [5]:
def q1_function(allocation_vector, distance_matrix):
    '''
    Calculates the total walking distance to pick products from shelves for all orders. 
    Also returns the distances covered in each order, the distances per order in decreasing order, 
    the indices and routes corresponding to each order where the distances are decreasing.
    '''
    OrdersByShelf = convert_orders_to_shelf_indices(allocation_vector)
    TotalDistance = 0           # initialise counter for total distance
    DistancesPerOrder = []      # initalise list to contain the distances for each order 
    routes = []                 # intialise list to contain the routes for each order
    for order in OrdersByShelf:
        visited_order_route, visited_order_dist = greedy_order_route(order, distance_matrix)
        routes.append(visited_order_route)
        DistancesPerOrder.append(visited_order_dist)
        TotalDistance += visited_order_dist

    # Replace DistancesPerOrder in the return statement with this if you want sorted distances to be returned
    SortedDistancesPerOrder = sorted(DistancesPerOrder, reverse=True)

    # The indices corresponding to the longest orders (descending order) 
    idx_longest_orders = sorted(range(len(DistancesPerOrder)), key=lambda i: DistancesPerOrder[i], reverse=True)

    # List of routes ordered by distance (descending)
    OrderedRoutes = [routes[i] for i in idx_longest_orders]
    
    return TotalDistance, DistancesPerOrder, SortedDistancesPerOrder, idx_longest_orders, OrderedRoutes

In [6]:
def getMostCommonProducts(Orders):
    '''
    This function takes the order matrix and extracts a list of all 
    product groups, ordered by the number of occurrences in the matrix.

    This lets us know which products are the most commonly ordered.
    '''
    flattenedList = [x for x in [item for sublist in Orders for item in sublist] if x != 0]

    itemCounts = Counter(flattenedList)

    sorted_items = sorted(itemCounts.items(), key=lambda x: x[1], reverse=True)
    original_numbers = [item for item, _ in sorted_items]

    return itemCounts, original_numbers


def getClosestShelves(Distances):
    distanceToDoor = Distances[0] # consider the distance vector for the door
    indexed_numbers = list(enumerate(distanceToDoor)) # indices and distances from door

    sorted_indexed_numbers = sorted(indexed_numbers, key=lambda x: x[1]) # sort indices by distance from door (ascending)
    sorted_indexes = [tuple([index,dist])  for index, dist in sorted_indexed_numbers if index > 0]

    return sorted_indexes # print index of shelf and its corresponding distance


def ConstructionHeuristic_mostCommonOrder(Orders, Distances, fill_extra_shelves = True, nbAdditionalShelves=6):
    mostCommonProducts = getMostCommonProducts(Orders)[1] # get the most common products

    # if statement to decide whether you want to fill extra shelves or not
    # nbAdditionalShelves determines how many extra shelves you fill
    if fill_extra_shelves == True:
        mostCommonProducts = mostCommonProducts + mostCommonProducts[:nbAdditionalShelves] 
    elif fill_extra_shelves == False:
        mostCommonProducts = mostCommonProducts

    ClosestShelves, ClosestDistances = map(list, zip(*getClosestShelves(Distances)))

    newShelves = [0]*96  # initialise empty shelf allocation vector

    for i, prod in zip(ClosestShelves, mostCommonProducts):
        newShelves[i-1] = prod  # assign products based on the closest shelves

    return np.array(newShelves)


def Random_Allocation_heuristic(fill_extra_shelves=True, nbAdditionalShelves=6):
    '''
    Constructive heuristic to assign products to shelves randomly.
    '''
    np.random.seed(123)
    prods_on_shelves = np.zeros(96, dtype = int)
    first_90_shelves = np.arange(1, 91) # assign the 90 products for the first time to the first 90 shelves
    np.random.shuffle(first_90_shelves) # randomly shuffle the products among the 90 shelves
    prods_on_shelves[:90] = first_90_shelves
    if fill_extra_shelves: # if we wish to fill some of the remaining empty shelves
        # randomly select nbAdditionalShelves number of products from the list of products and assign these to the empty shelves
        prods_on_shelves[90:90+nbAdditionalShelves] = np.random.choice(np.arange(1, 91), size = nbAdditionalShelves, replace=False)

    return prods_on_shelves




def addTemporaryFarAwayShelves(Distances):
    '''
    Adds temporary shelves that are far away from everything else to the distances matrix
    '''
    number_to_fill = 100

    # Number of rows and columns to add
    num_rows_to_add = 90
    num_cols_to_add = 90

    # Create a new row filled with the specified number
    new_row = np.full((1, Distances.shape[1]), number_to_fill)

    # Repeat the new row to match the number of rows to add
    new_rows = np.repeat(new_row, num_rows_to_add, axis=0)

    # Concatenate the existing array with the new rows
    result_array = np.concatenate((Distances, new_rows), axis=0)

    new_column = np.full((result_array.shape[0], 1), number_to_fill)
    new_columns = np.repeat(new_column, num_cols_to_add, axis=1)

    result_array = np.concatenate((result_array, new_columns), axis=1)

    return result_array


def GreedyHeuristic(Distances):
    ''' 
    Greedy heuristic to construct an initial solution by adding the products to shelf one at a time and checking
    distances.
    '''
    refilledProducts = False
    tempProducts = np.arange(1, 91).tolist()
    tempDistances = addTemporaryFarAwayShelves(Distances) 

    normalShelves = [0]*96
    extraShelves = np.arange(1, 91).tolist()

    tempAllocations = normalShelves.copy() + extraShelves.copy() # starting allocation
    orderedShelves, closest_distances = map(list, zip(*getClosestShelves(Distances))) # shelves closest to door

    for o_shelf in orderedShelves:
        shelf = o_shelf-1
        mostReduced = 1e9
        bestP = None
        
        tempShelves = tempAllocations.copy()
        baseline = q1_function(tempShelves, tempDistances)[0]

        for p in tempProducts:
            tempShelves = tempAllocations.copy()
            tempShelves[shelf] = p

            newobj = q1_function(tempShelves, tempDistances)
            reduction = newobj[0] - baseline

            if reduction <= mostReduced:
                bestP = p
                mostReduced = reduction

        tempAllocations[shelf] = bestP
        tempProducts.remove(bestP)

        if refilledProducts == False:
            indexOfBestP_onExtraShelf = tempAllocations[96:].index(bestP)
            del tempAllocations[indexOfBestP_onExtraShelf+96]

        if len(tempProducts) == 0:
            tempProducts = [i for i in range(1, 91)]
            refilledProducts = True

        # print(f"Assigned product {bestP} -> shelf {shelf+1}")
    return tempAllocations

# Writing allocation vectors to txt files

In [7]:
def allocation_to_txt(allocation_vector, file_path):
    np.savetxt(file_path, allocation_vector, fmt = "%d", delimiter = ",", newline=" ")

In [8]:
mostCommonOrderAllocation = ConstructionHeuristic_mostCommonOrder(Orders, 
                                                                  FullDistanceMatrix, 
                                                                  fill_extra_shelves=False)
allocation_to_txt(mostCommonOrderAllocation, "Allocation_Vectors/CommonOrder_0_Extra.txt")
allocation_to_txt(Random_Allocation_heuristic(nbAdditionalShelves=0), "Allocation_Vectors/Random_0_Extra.txt")

In [10]:
allocation_to_txt(GreedyHeuristic(FullDistanceMatrix), "Allocation_Vectors/GreedyAllocation.txt")

In [11]:
for i in range(6):
    allocation_extra_shelves = ConstructionHeuristic_mostCommonOrder(Orders,
                                                                     FullDistanceMatrix,
                                                                     fill_extra_shelves=True,
                                                                     nbAdditionalShelves=i+1)
    allocation_to_txt(allocation_extra_shelves, f"Allocation_Vectors/CommonOrder_{i+1}_Extra.txt")
    allocation_to_txt(Random_Allocation_heuristic(nbAdditionalShelves=i+1), f"Allocation_Vectors/Random_{i+1}_Extra.txt")

In [12]:
for j in range(1,5):
    distance_matrix = pd.read_excel(f"Distance_Matrices/DistanceMatrix_{j}_aisles.xlsx", 
                                    sheet_name = "DistanceMatrixMetres", 
                                    header=None, skiprows= 1)
    allocation_0_extra = ConstructionHeuristic_mostCommonOrder(Orders, distance_matrix, fill_extra_shelves=False)
    allocation_6_extra = ConstructionHeuristic_mostCommonOrder(Orders, distance_matrix)
    allocation_to_txt(allocation_0_extra, f"Allocation_Vectors/CommonOrder_0extra_{j}mid_aisle.txt")
    allocation_to_txt(allocation_6_extra, f"Allocation_Vectors/CommonOrder_6extra_{j}mid_aisle.txt")

In [ ]:
for j in range(1,5):
    start_time = time.time()
    distance_matrix = pd.read_excel(f"Distance_Matrices/DistanceMatrix_{j}_aisles.xlsx", 
                                    sheet_name = "DistanceMatrixMetres", 
                                    header=None, skiprows= 1)
    greedy_alloc = GreedyHeuristic(distance_matrix)
    allocation_to_txt(greedy_alloc, f"Allocation_Vectors/Greedy_{j}mid_aisle.txt")
    print(f"saved file for {j} middle aisles in {time.time() - start_time} seconds.")

saved file for 1 middle aisles in 553.6866328716278 seconds.
saved file for 2 middle aisles in 483.5562000274658 seconds.
saved file for 3 middle aisles in 547.9849553108215 seconds.
saved file for 4 middle aisles in 622.1526012420654 seconds.
